In [1]:
import numpy as np
import pandas as pd

from sklearn.decomposition import KernelPCA, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from src.features.build_features import StockTechnicals

In [2]:
# Load training data using MSFT
train_ticker = "MSFT"
all_msft_data = pd.read_csv(f'../data/{train_ticker}.csv')

# number of future days for SMA to rise
N = 26


In [3]:
technicals = StockTechnicals(all_msft_data)
X = technicals.features
y = technicals.future_sma_higher_than_current_price(N)

# we don't have the last N days of data (they're in the future)
y = y[~np.isnan(y)]
X = X[:len(y)]

/Users/ted/Library/Caches/pypoetry/virtualenvs/njit-ml-scratch-l36YVstm-py3.7/lib/python3.7/site-packages/ta/trend.py:567: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/Users/ted/Library/Caches/pypoetry/virtualenvs/njit-ml-scratch-l36YVstm-py3.7/lib/python3.7/site-packages/ta/trend.py:571: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])
/Users/ted/Documents/NJIT/675_MachineLearning/proj/njit-ml-project-spring-2020/src/features/build_features.py:59: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  slope, _ = np.linalg.lstsq(A, y)[0]


In [4]:
# limit to top features as determined by a RandomForest in feature-selection.ipnyb
top_features = [
    'trend_visual_ichimoku_b',
    'volume_obv',
    'volatility_kcw',
    'volatility_atr',
    'trend_mass_index',
    'trend_kst_sig',
    'volume_cmf',
    'trend_adx',
    'trend_macd_signal',
    'volatility_bbw',
    'trend_kst_diff',
    'momentum_tsi',
    'trend_macd_diff',
]

In [5]:
# limit X to top features only
X_top = X[top_features]
X_train, X_test, y_train, y_test = train_test_split(X_top, y, random_state=2, stratify=y)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [6]:
# run a logistic regression to get a benchmark performance
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std, y_train)
    y_pred = lr.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")
    

Accuracy score: 0.649932157394844
Accuracy score: 0.6377204884667571
Accuracy score: 0.6431478968792401
Accuracy score: 0.6417910447761194
Accuracy score: 0.6417910447761194


In [ ]:
# Limiting to the top 13 features achieves an accuracy very close to using all 69 (~65% v ~69%).
# As an alternative, let's see what happens when reducing to 13 "fake" features using PCA 

In [ ]:
# set up test and train datasets using full X
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, stratify=y)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# fit a pca with 13 "fake" attributes
pca = PCA(13, random_state=1, tol=0.1)
X_train_std_pca = pca.fit_transform(X_train_std)
X_test_std_pca = pca.transform(X_test_std)

In [ ]:
# run a logistic regression with the 13-feature pca dataset
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std_pca, y_train)
    y_pred = lr.predict(X_test_std_pca)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")

In [ ]:
# using a PCA with 13 "fake" attributes doesn't acheive better results than simply using
# our 13 top attributes as selected from the RandomForest. Therefore, we'll stick with those
# as they're more easily explained. Next we'll try using the kernel trick to see if we can
# improve classification by temporarily projecting data into a higher dimensional space

In [5]:
# trying out kpca
kpca = KernelPCA(13, random_state=1, kernel='rbf')
# kpca = PCA(13, random_state=1)
X_train_std = kpca.fit_transform(X_train_std)
X_test_std = kpca.transform(X_test_std)

# run a logistic regression
for c in [0.01, 0.1, 1.0, 10, 100]:
    lr = LogisticRegression(C=c, random_state=2, solver='liblinear')
    lr.fit(X_train_std, y_train)
    y_pred = lr.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")

Accuracy score: 0.6350067842605156
Accuracy score: 0.6377204884667571
Accuracy score: 0.6621438263229308
Accuracy score: 0.6485753052917232
Accuracy score: 0.6472184531886025


In [6]:
from sklearn.neural_network import MLPClassifier

for a in [0.1, 1.0, 10, 100]:
    mlp = MLPClassifier(hidden_layer_sizes=(39, 39), solver='lbfgs', max_iter=1000, random_state=2, alpha=a, activation='tanh')
    mlp.fit(X_train_std, y_train)
    y_pred = mlp.predict(X_test_std)

    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")


/Users/ted/Library/Caches/pypoetry/virtualenvs/njit-ml-scratch-l36YVstm-py3.7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/ted/Library/Caches/pypoetry/virtualenvs/njit-ml-scratch-l36YVstm-py3.7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy score: 0.8046132971506106
Accuracy score: 0.8046132971506106
Accuracy score: 0.6526458616010855
Accuracy score: 0.6350067842605156
